In [1]:
#Needed in general
import sys

#Quick fudge to make import from ../Scripts work
# sys.path.append('../../../common_code')
# from PostProcessingScripts import * 
sys.path.append('../../../common_code/KDEpy')
from KDEpy import FFTKDE 
# from ClassFormationChannels_5mainchannels import * 

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



colors = ['#1f77b4', '#ff7f0e']
colors_lighter = [adjust_lightness(color=colors[0], amount=2.2),adjust_lightness(color=colors[1], amount=1.7)]

from formation_channels import * 
# Generate a distribution and draw 2**6 data points
from scipy import stats


ModuleNotFoundError: No module named 'cutils'

In [2]:
import KDEpy

ModuleNotFoundError: No module named 'cutils'

In [ ]:
import cutils

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,8))





channelColorDict = {'classic':'#118AB2', 'stable B no CEE':'orange',  'immediate CE': '#EF476F'  , r'double-core CE':'#073B4C', 'other':'gray', 'vi':'cyan', 'vii':'#FFD166'}
List_formationchannelOptions = ['All',  'classic',  'stable B no CEE',  'immediate CE',  r'double-core CE', 'vi', 'vii', 'other']
ind_formationchannelOptions = [7,  1, 2, 3, 4, 5, 6, 0]
dictFormationChannelIndex =  {List_formationchannelOptions[i]: ind_formationchannelOptions[i] for i in range(len(List_formationchannelOptions))}



# channelColorDict = {1:'#118AB2', 2:'#EF476F', 3:'#FFD166', 4:'#073B4C', 0:'gray'}
channelColorDict_lighter = {'classic':adjust_lightness(color='#118AB2', amount=1.6),'stable B no CEE':adjust_lightness(color='orange', amount=1.4), 'immediate CE':adjust_lightness(color='#EF476F', amount=1.2),\
                            r'double-core CE':adjust_lightness(color='#073B4C', amount=1.8), 'other':adjust_lightness(color='gray', amount=1.5),  'vi':adjust_lightness(color='cyan', amount=1.5), 'vii':adjust_lightness(color='#FFD166', amount=1.2)}
channelList = ['classic', 'stable B no CEE', 'vii',  'immediate CE',  r'double-core CE', 'other'] #, 'vi']

for nrC, Channel in enumerate(channelList): 

    c_FC = channelColorDict[Channel]
    colors_lighter_FC =  adjust_lightness(color=c_FC, amount=1.6)
    
    plt.scatter([nrC], [nrC], s=1000, c=c_FC)
    plt.scatter([nrC], [nrC+1], s=1000, c=channelColorDict_lighter[Channel])
    
plt.show()
    
    
    
    


In [ ]:




def plot_FC_distribution(axe='None', xparam='chiEff', BPSmodelName='A', mode='pdf',\
                          spin_threshold='None', bw=0.01, xlim=[0,1], ylim=[0,1],\
                          plotYlog='False', ylim_threshold=0.02):#, mssfr='112'):
    
    fs_l = 28 # label fontsize
    
#     colors = ['#1f77b4', '#ff7f0e']
#     colors_lighter = [adjust_lightness(color=colors[0], amount=2.2),adjust_lightness(color=colors[1], amount=1.7)]

    DCOtype='BBH'
    # path for files 
    path_ = '/Volumes/Andromeda2/DATA/AllDCO_bugfix/' + alphabetDirDict[BPSmodelName] +'/'
    path  = path_ + 'COMPASCompactOutput_'+ DCOtype +'_' + BPSmodelName + '.h5'
    
    fdata = h5.File(path, 'r')
    massCO_ZAMSM1 = fdata['doubleCompactObjects']['M1'][...].squeeze()
    massCO_ZAMSM2 = fdata['doubleCompactObjects']['M2'][...].squeeze()
    # M1 will be the most massive, M2 the least massive compact object. 
    massCO_LVKM1, massCO_LVKM2 = obtainM1BHandM2BHassymetric(m1=fdata['doubleCompactObjects']['M1'][...].squeeze(), m2=fdata['doubleCompactObjects']['M2'][...].squeeze()) 
    MassRatioCO_LVK = massCO_LVKM2/massCO_LVKM1
    
    seedsDCO  = fdata['doubleCompactObjects']['seed'][...].squeeze()  # get the seeds in the DCO file 
    seedsSN   = fdata['supernovae']['randomSeed'][...].squeeze()    # get the seeds in the SN file 
    indices   = np.sort(np.unique(seedsSN[1::2], return_index=True)[1])
    maskSNdco = np.in1d(seedsSN,  seedsDCO) # mask in the SNe files the SNe that correspond to our DCO
    whichSN   = fdata['supernovae']['whichStar'][...].squeeze()[maskSNdco]  # this is 1 if the initially primary star goes SN and 2 if the secondary goes supernova     
    whichSN2  = whichSN[1::2][indices]

    # either SN2 = primary (1) and M1 is > M2, or SN2 = secondary & M1 < M2 
    # this takes into account (first term) rejuvenation 
    mask_MRR = ((whichSN2==1) & (massCO_ZAMSM1>massCO_ZAMSM2) ) | ((whichSN2==2) & (massCO_ZAMSM1<massCO_ZAMSM2)) 

    
    
    # obtain formation channels 
    seeds = fdata['doubleCompactObjects']['seed'][...].squeeze()
    channels = identify_formation_channels(seeds=seeds, file=fdata)
    
    
    
    del massCO_ZAMSM1
    del massCO_ZAMSM2
    del whichSN2
    del whichSN
    del maskSNdco
    del indices
    del seedsSN
    del seedsDCO


    
    if (mode in ['spin_PDF', 'spin_fraction', 'spinOne_fraction', 'spinTwo_fraction', 'm1spin_or_m2spin_fraction', 'spin_CDF' ]) | (xparam in ['chi_of_spinning_BH', 'chi_effective', 'log10_t_delay']):
        spin = COspin(data_path=path, state='he_depletion')  # set class 
        spin.setCOMPASData() # reads in the COMPAS DCO parameters 
        spinZAMSM1, spinZAMSM2  = spin.BaveraSpin() #ZAMS M1 SPIN 

        spinLVKM1, spinLVKM2 = np.zeros_like(spinZAMSM1), np.zeros_like(spinZAMSM1)
        spinLVKM1[mask_MRR] = spinZAMSM2[mask_MRR]  # MRR so M1 comes from M2ZAMS, we assign it spin from M2ZAMS
        spinLVKM1[~mask_MRR] = spinZAMSM1[~mask_MRR]  # no MRR so M1 comes from M1ZAMS, we assign it spin from M1ZAMS
        spinLVKM2[mask_MRR] = spinZAMSM1[mask_MRR]   # MRR so M2 comes from M1ZAMS, we assign it spin from M1ZAMS
        spinLVKM2[~mask_MRR] = spinZAMSM2[~mask_MRR]   # no MRR so M2 comes from M2ZAMS, we assign it spin from M2ZAMS     

        # spin_threshold = 0.05 # definition of "spinning BH"
        mask_LVKM1_spinning = (spinLVKM1 > spin_threshold ) 
        mask_LVKM2_spinning = (spinLVKM2 > spin_threshold ) # definition of "spinning BH"
        mask_anySpin = (spinLVKM1 > spin_threshold ) | (spinLVKM2 > spin_threshold )    

    

    if xparam=='chirp_mass_LVK':
        param_x = chirpmass(massCO_LVKM1, massCO_LVKM2)
        nameX = r'$\mathcal{M}_{\rm{c}} \ [M_{\odot}]$'
        nameY = r'\textbf{PDF}'
        xx = np.linspace(1,100,1000)
        print(param_x)
        print(np.shape(param_x))
        
    elif xparam=='mass_ratio_LVK':
        param_x = MassRatioCO_LVK
        nameX = r'$q$'
        nameY = r'\textbf{PDF}'
        xx = np.linspace(-0.2,1.2,1000)

        

    elif xparam=='mass_1_LVK':
        param_x = massCO_LVKM1
        nameX = r'$m_1 [M_{\odot}]$'
        nameY = r'\textbf{PDF}'
        xx = np.linspace(-1,1000,1000)
        print(param_x)
        print(np.shape(param_x))
        
    elif xparam=='chi_of_spinning_BH':
        param_x = spinLVKM1 + spinLVKM2

        nameX = r'$\chi_{\rm{i}}$'
        nameY = r'\textbf{PDF}'
        xx = np.linspace(-0.2,2,1000)  

        
    elif xparam=='chi_effective':
        param_x = ((spinLVKM1*massCO_LVKM1) + (spinLVKM2*massCO_LVKM2)) / (massCO_LVKM1+massCO_LVKM2)
        nameX = r'$\chi_{\rm{eff}}$'
        nameY = r'\textbf{PDF}'
        xx = np.linspace(-0.2,2,1000)          
        
    elif xparam=='log10_t_delay':
        param_x = (fdata['doubleCompactObjects']['tform'][...].squeeze() +  fdata['doubleCompactObjects']['tc'][...].squeeze() ) / 1000 # divide by 1000 to make it in [Gyr]
        param_x = np.log10(param_x)
        nameX = r'$t_{\rm{delay}} \ [\rm{Gyr}]$'
        nameY = r'\textbf{PDF}'  
        xx = np.linspace(-4,2,1000)  

    
    if (mode=='MRR_PDF') | (mode=='spin_PDF'):
        nameY = r'\textbf{PDF}'
    elif (mode=='MRR_fraction'):
        nameY = r'$\rm{f}_{\rm{channel}}$'
    elif (mode=='notMRR_fraction'):
        nameY = r'$\rm{f}_{\rm{non MRR}}$'
    elif (mode=='spin_fraction'):
        nameY =   r'$\rm{f}_{\chi_{1} > %s}$'%spin_threshold
    elif (mode=='m1spin_or_m2spin_fraction'):
        nameY =   r'$\rm{f}_{\chi_{\rm{i}} > %s}$'%spin_threshold 
    elif (mode=='MRR_CDF') | (mode=='spin_CDF'):
        nameY = r'\textbf{CDF}'
    elif (mode=='spinOne_fraction'):
        nameY =   r'$\rm{f}_{\chi_{1} > %s}$'%spin_threshold
    elif (mode=='spinTwo_fraction'):
        nameY =   r'$\rm{f}_{\chi_{2} > %s}$'%spin_threshold
    else:
        raise ValueError("the provided `mode` is not recognized, the value given is %s"%mode)
        
    
    estimator = FFTKDE(kernel='biweight', bw=bw)      
    
    if plotYlog==True:       
        axe.set_yscale('log')
        
        
        
#     channelList = ['classic', 'stable B no CEE', 'vii',  'immediate CE',  r'double-core CE', 'other']
    adjustedChannelList = ['classic', 'stable B no CEE', 'immediate CE',  r'double-core CE', 'other']
#     adjustedChannelList = ['classic']#, 'stable B no CEE', 'immediate CE']#
    for nrC, Channel in enumerate(adjustedChannelList): 
        
        print('now at channel: ', Channel)
        ind_wanted = dictFormationChannelIndex[Channel]
        
        
        
        c_FC = channelColorDict[Channel]
        colors_lighter_FC =  channelColorDict_lighter[Channel]
#         print('now at channel', Channel)




        mask_MRR = (channels==ind_wanted)
    
        if len(mask_MRR)>0:
    
#             for ind_mssfr, mssfr in enumerate(MSSFRnameslist[0:]):
            for ind_mssfr, mssfr in enumerate(MSSFRnameslist[0:2]):    

                ls_ = '-'
                Highlight = False
                if ((mssfr in ['123']) & (BPSmodelName in ['K'])):
                    ls_ = ':'
                    Highlight = True
                elif  ((mssfr in ['312']) & (BPSmodelName in ['T'])):
                    ls_ = '--'
                    Highlight = True
                elif ((mssfr in ['231']) & (BPSmodelName in ['O'])):
                    ls_ = '-.'
                    Highlight = True




                ### read in MSSFR weights: ###
    #             fparam_key = 'weights_detected'
                fparam_key = 'weights_intrinsic'
                weightheader = 'w_' + mssfr
                weights_ = fdata[fparam_key][weightheader][...].squeeze()
                w = weights_
                bandwidth=1

                if (mode=='MRR_PDF') | (mode=='MRR_fraction') |  (mode=='notMRR_fraction') : 
                    # we want at least 10 datapoint for KDE
                    if len(w[mask_MRR])>10:
                
                        yy_MRR = estimator.fit(param_x[mask_MRR], weights=w[mask_MRR]).evaluate(xx) 
                        yy_total = estimator.fit(param_x, weights=w).evaluate(xx) 
                        rel_weight_MRR    = np.sum(w[mask_MRR])  / (np.sum(w))
                        yy_MRR *= rel_weight_MRR
                        rel_weight_MRR    = np.sum(w[mask_MRR])  / (np.sum(w))
                        
                        
#                         KDE = gaussian_kde(param_x[mask_MRR], weights=w[mask_MRR], bw_method = bandwidth)
#                         yy_MRR = KDE(xx)
#                         rel_weight_MRR    = np.sum(w[mask_MRR])  / (np.sum(w))
#                         yy_MRR *= rel_weight_MRR
                        # cheap histogram 
#                         KDE = gaussian_kde(param_x[mask_MRR], weights=w[mask_MRR], bw_method = bandwidth)
#                         yy_MRR = KDE(xx)
                        rel_weight_MRR    = np.sum(w[mask_MRR])  / (np.sum(w))
#                         yy_MRR *= rel_weight_MRR


# #                         bins = np.arange(0,50, 2)
#                         norm = 1
#                         bins=xx
#                         hist, bin_edge = np.histogram(param_x[mask_MRR], weights = w[mask_MRR], bins=xx)
#                         center_bins = (bin_edge[:-1] + bin_edge[1:])/2.
#                         # And the KDE
#                         kernel = stats.gaussian_kde(param_x[mask_MRR], bw_method='scott', weights=w[mask_MRR])
#                         binwidth = np.diff(bin_edge) 
#                         x_KDE = np.arange(bins[0],bins[-1],0.1)
#                         KDEy_vals = kernel(x_KDE)*sum(hist)*np.diff(bins)[0]/norm #re-normalize the KDE
                        
                    else:
                        print('%s data points for KDE, this is below the threshold 10, not drawing KDE'%len(w[mask_MRR]))

#                 elif mode in ['spin_PDF', 'spin_fraction', 'spinOne_fraction', 'spinTwo_fraction']:
#     #                 yy_nonMRR = estimator.fit(param_x[mask_LVKM2_spinning], weights=w[mask_LVKM2_spinning]).evaluate(xx)
#                     yy_MRR = estimator.fit(param_x[mask_LVKM1_spinning], weights=w[mask_LVKM1_spinning]).evaluate(xx)
#                     yy_total = estimator.fit(param_x, weights=w).evaluate(xx)

#     #                 rel_weight_nonMRR = np.sum(w[mask_LVKM2_spinning]) / (np.sum(w))
#                     rel_weight_MRR    = np.sum(w[mask_LVKM1_spinning])  / (np.sum(w))
#     #                 yy_nonMRR *= rel_weight_nonMRR
#                     yy_MRR *= rel_weight_MRR


#                 ## IF WE ARE PLOTTING CDFs
#                 elif (mode=='MRR_CDF') | (mode=='spin_CDF'):
#                     if Highlight==True:
#     #                     axe.hist(param_x[~mask_MRR], bins=10000, color=colors[0], lw=3, zorder=15, alpha=1, ls=ls_ , density=True, weights=weights_[~mask_MRR], cumulative=True, histtype='step')
#                         axe.hist(param_x[mask_MRR],  bins=10000, color= c_FC, lw=3, zorder=16, alpha=1, ls=ls_,   density=True, weights=weights_[mask_MRR],  cumulative=True, histtype='step')
#                     else:
#     #                     axe.hist(param_x[~mask_MRR], bins=10000, color=colors_lighter[0], lw=1.5, zorder=5, alpha=1, ls=ls_, density=True, weights=weights_[~mask_MRR], cumulative=True, histtype='step')
#                         axe.hist(param_x[mask_MRR],  bins=10000, color=colors_lighter_FC, lw=1.5, zorder=6, alpha=1, ls=ls_,  density=True, weights=weights_[mask_MRR],  cumulative=True, histtype='step')

#                     axe = layoutAxes(axe, nameX=nameX, nameY=nameY, setMinor=False, labelpad=5, fontsize=fs_l, labelSizeMajor=fs_major)                        


                ### PLOTTING ###
                ## IF WE ARE PLOTTING PDFs
                if (mode=='MRR_PDF') | (mode=='spin_PDF'):

                        #                     axe.plot(xx, yy_nonMRR, color=colors[0], lw=3, zorder=15, alpha=1, ls=ls_)
                    norm = 1.
#                     axe.bar(center_bins, hist/norm, width= np.diff(bins), alpha=1.0, fill=False, edgecolor=c_FC, lw = 1., zorder = 0) 
# #                         axe.plot(xx, yy_MRR,    color=c_FC, lw=3, zorder=16, alpha=1, ls=ls_)
#                     axe.plot(x_KDE, KDEy_vals, lw=5, color=c_FC, zorder =1)
#                     axe.fill_between(x_KDE, y1=0, y2=KDEy_vals, color=colors_lighter_FC, alpha = 0.05, zorder = 1)

#                     axe = layoutAxes(axe, nameX=nameX, nameY=nameY, setMinor=False, labelpad=0, fontsize=fs_l, labelSizeMajor=fs_major)  


                # PLOTTING FRACTION     
#                 elif mode in ['MRR_fraction', 'spin_fraction',  'notMRR_fraction', 'spinOne_fraction', 'spinTwo_fraction']: 

#     #                 yy_total = (yy_nonMRR + yy_MRR)
#                     # in some cases the overall PDF is so small, that the ratio is dominated by some shot noise. So we make those zero. 
#                     mask_too_small_values =  (yy_MRR < ylim_threshold)
#                     yy_MRR[mask_too_small_values] = np.zeros(int(np.sum(mask_too_small_values)))
#     #                 yy_nonMRR[mask_too_small_values] = np.zeros(int(np.sum(mask_too_small_values)))

#                     if (mode=='MRR_fraction') | (mode=='spin_fraction') | (mode=='spinOne_fraction'):
#                         ratio = yy_MRR/yy_total

#     #                 elif (mode=='notMRR_fraction') | (mode=='spinTwo_fraction'): 
#     #                     ratio = yy_nonMRR/yy_total
#     #                     color = colors[0]
#     #                     color_lighter = colors_lighter[0]

#                     if Highlight==True:
#                         axe.plot(xx, ratio,     color=c_FC,          lw=3,  zorder=20, alpha=1, ls=ls_)
#                     else:
#                         axe.plot(xx, ratio,     color=colors_lighter_FC,  lw=1.5, zorder=6, alpha=1, ls=ls_)

#                     # remove labels and ticks  on X axe
#                     axe = layoutAxesNoXlabel(axe, nameX=nameX, nameY=nameY, setMinor=False, labelpad=5, fontsize=fs_l, labelSizeMajor=fs_major)
#                     for tick in axe.xaxis.get_major_ticks():
#                         tick.tick1line.set_visible(False)
#                         tick.tick2line.set_visible(False)
#                         tick.label1.set_visible(False)
#                         tick.label2.set_visible(False)


#                 elif (mode=='m1spin_or_m2spin_fraction'):

#                     yy_total = estimator.fit(param_x, weights=w).evaluate(xx)
#                     yy_anySpin = estimator.fit(param_x[mask_anySpin], weights=w[mask_anySpin]).evaluate(xx)


#                     # in some cases the overall PDF is so small, that the ratio is dominated by some shot noise. So we make those zero. 
#                     mask_too_small_values = (yy_anySpin < ylim_threshold) & (yy_total < ylim_threshold)
#                     yy_anySpin[mask_too_small_values] = np.zeros(int(np.sum(mask_too_small_values)))            


#                     if Highlight==True:
#                         axe.plot(xx, yy_anySpin*np.sum(w[mask_anySpin]) / (yy_total*np.sum(w)),     color='k',        lw=2, zorder=20, alpha=1, ls=ls_)
#                     else:
#                         axe.plot(xx, yy_anySpin*np.sum(w[mask_anySpin]) / (yy_total*np.sum(w)),     color='lightgray',      lw=2, zorder=6, alpha=1, ls=ls_)


#                     # remove labels and ticks  on X axe
#                     axe = layoutAxesNoXlabel(axe, nameX=nameX, nameY=nameY, setMinor=False, labelpad=5, fontsize=fs_l, labelSizeMajor=fs_major)
#                     for tick in axe.xaxis.get_major_ticks():
#                         tick.tick1line.set_visible(False)
#                         tick.tick2line.set_visible(False)
#                         tick.label1.set_visible(False)
#                         tick.label2.set_visible(False)

    if mode in [ 'MRR_fraction', 'notMRR_fraction', 'spinTwo_fraction', 'spinOne_fraction', 'm1spin_or_m2spin_fraction']:
        axe.set_yticks([ 0.5, 1])
    elif mode in ['MRR_CDF', 'spin_CDF']:
        axe.set_yticks([0.5, 0.75, 1])
        

            
    axe.set_xlim(xlim[0], xlim[1])
    axe.set_ylim(ylim[0], ylim[1])
    axe.grid(True)
    
    


    
    return axe 









## ONE PLOT:  Loop over BPS models:

In [ ]:
## writing here 






# for ind_BPS, BPSmodel in enumerate(BPSnameslist[:]):
# just one model 
for ind_BPS, BPSmodel in enumerate([BPSnameslist[0]]):

    fig, ax = plt.subplots(1,1, figsize=(14,9))#,\

    fs_major=34
    
    print('running model %s'%BPSmodel)
    
    s_text = r'model %s: '%BPSmodel + alphabetPhysicalNameDictWithEnter[BPSmodel]
    ax.text(0.98, 0.97, s=s_text , rotation = 0, fontsize=24, color = 'k', va='top', ha = 'right',transform=ax.transAxes)#, weight = 'bold')

    # CHIRP MASS 
    print('at mass_1_LVK')
#     print('at chirp mass')
    bw=0.85
    ylim_threshold = 0.0004       
    xparam = 'mass_1_LVK'
#     xparam = 'chirp_mass_LVK'
    plotYlog = False
    xlim, ylim = [0, 48], [0.0, 2.03]

    plotYlog = True
    ylim = [0.0003, 0.15]
    ax = plot_FC_distribution(axe=ax, xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_PDF',\
                                  spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)


    xx = np.linspace(1000,20000,100)
    yy = np.linspace(1000,20000,100)
    lw=12
    fs = 30

    # ax.plot(xx, yy,    color=colors_lighter[1], lw=lw, zorder=16, alpha=1, label=r'\textbf{MRR}')#, ls=ls_)
    # ax[2,1].plot(xx, yy, color=colors_lighter[0], lw=lw, zorder=15, alpha=1, label=r'\textbf{non-MRR}')#, ls=ls_)
    # ax[2,1].legend(fontsize=fs, frameon=False, loc='upper left')

    plt.tight_layout()  
    # plt.subplots_adjust(wspace=0.2, hspace=0.1)

#     plt.savefig('./singlemodel/super_FC_split_panel_%s.pdf'%BPSmodel, transparent=False, bbox_inches="tight",  format='pdf')
#     plt.savefig('./singlemodel/super_FC_split_panel_%s.png'%BPSmodel, transparent=False, bbox_inches="tight", dpi=600, format='png')
    plt.show()  










In [ ]:
class FFTKDE(BaseKDE):
    r"""
    This class implements a convolution (FFT) based computation of a KDE.
    While this implementation is very fast, there are some limitations: (1) the
    bandwidth must be constant, (2) the KDE must be evaluated on an
    equidistant grid and (3) the grid must encompass every data point.
    The finer the grid, the smaller the error.

    The evaluation step is split into two phases. First the :math:`N` data
    points are binned using a linear binning routine on an equidistant grid `x`
    with :math:`n` grid points. This runs in :math:`O(N 2^d)` time.
    Then the kernel is evaluated once on :math:`\leq n` points and the result
    of the kernel evaluation and the binned data is convolved. Using the
    convolution theorem, this step runs in :math:`O(n \log n)` time.
    While :math:`N` may be millions, :math:`n` is typically 2**10. The total
    running time of the algorithm is :math:`O(N 2^d + n \log n)`.
    See references for more information.

    The implementation is reminiscent of the one found in statsmodels. However,
    unlike the statsmodels implementation every kernel is available for FFT
    computation, weighted data is available for FFT computation, and no large
    temporary arrays are created.

    Parameters
    ----------
    kernel : str
        The kernel function. See cls._available_kernels.keys() for choices.
    bw : float or str
        Bandwidth or bandwidth selection method. If a float is passed, it
        is the standard deviation of the kernel. If a string it passed, it
        is the bandwidth selection method, see cls._bw_methods.keys() for
        choices.

    Examples
    --------
    >>> data = np.random.randn(2**10)
    >>> # (1) Automatic bw selection using Improved Sheather Jones (ISJ)
    >>> x, y = FFTKDE(bw='ISJ').fit(data).evaluate()
    >>> # (2) Explicit choice of kernel and bw (standard deviation of kernel)
    >>> x, y = FFTKDE(kernel='triweight', bw=0.5).fit(data).evaluate()
    >>> weights = data + 10
    >>> # (3) Using a grid and weights for the data
    >>> y = FFTKDE(kernel='epa', bw=0.5).fit(data, weights).evaluate(x)
    >>> # (4) If you supply your own grid, it must be equidistant
    >>> y = FFTKDE().fit(data)(np.linspace(-10, 10, num=2**12))

    References
    ----------
    - Wand, M. P., and M. C. Jones. Kernel Smoothing.
      London ; New York: Chapman and Hall/CRC, 1995. Pages 182-192.
    - Statsmodels implementation, at
      ``statsmodels.nonparametric.kde.KDEUnivariate``.

    """

    def __init__(self, kernel="gaussian", bw=1, norm=2):
        self.norm = norm
        super().__init__(kernel, bw)
        assert isinstance(self.norm, numbers.Number) and self.norm > 0

def fit(self, data, weights=None):
        """
        Fit the KDE to the data. This validates the data and stores it.
        Computations are performed upon evaluation on a specific grid.

        Parameters
        ----------
        data: array-like
            The data points.
        weights: array-like
            One weight per data point. Must have same shape as the data.

        Returns
        -------
        self
            Returns the instance.

        Examples
        --------
        >>> data = [1, 3, 4, 7]
        >>> weights = [3, 4, 2, 1]
        >>> kde = FFTKDE().fit(data, weights=None)
        >>> kde = FFTKDE().fit(data, weights=weights)
        >>> x, y = kde.evaluate()
        """

        # Sets self.data
        super().fit(data, weights)
        return self


def evaluate(self, grid_points=None):
        """
        Evaluate on equidistant grid points.

        Parameters
        ----------
        grid_points: array-like, int, tuple or None
            A grid (mesh) to evaluate on. High dimensional grids must have
            shape (obs, dims). If an integer is passed, it's the number of grid
            points on an equidistant grid. If a tuple is passed, it's the
            number of grid points in each dimension. If None, a grid will be
            automatically created.

        Returns
        -------
        y: array-like
            If a grid is supplied, `y` is returned. If no grid is supplied,
            a tuple (`x`, `y`) is returned.

        Examples
        --------
        >>> kde = FFTKDE().fit([1, 3, 4, 7])
        >>> # Three ways to evaluate a fitted KDE object:
        >>> x, y = kde.evaluate()  # (1) Auto grid
        >>> x, y = kde.evaluate(256)  # (2) Auto grid with 256 points
        >>> # (3) Use a custom grid (make sure it's wider than the data)
        >>> x_grid = np.linspace(-10, 25, num=2**10)  # <- Must be equidistant
        >>> y = kde.evaluate(x_grid)  # Notice that only y is returned
        """

        # This method sets self.grid_points and verifies it
        super().evaluate(grid_points)

        # Extra verification for FFTKDE (checking the sorting property)
        if not grid_is_sorted(self.grid_points):
            raise ValueError("The grid must be sorted.")

        if isinstance(self.bw, numbers.Number) and self.bw > 0:
            bw = self.bw
        else:
            raise ValueError("The bw must be a number.")
        self.bw = bw

        # Step 0 - Make sure data points are inside of the grid
        min_grid = np.min(self.grid_points, axis=0)
        max_grid = np.max(self.grid_points, axis=0)

        min_data = np.min(self.data, axis=0)
        max_data = np.max(self.data, axis=0)
        if not ((min_grid < min_data).all() and (max_grid > max_data).all()):
            raise ValueError("Every data point must be inside of the grid.")

        # Step 1 - Obtaining the grid counts
        # TODO: Consider moving this to the fitting phase instead
        data = linear_binning(self.data, grid_points=self.grid_points, weights=self.weights)

        # Step 2 - Computing kernel weights
        g_shape = self.grid_points.shape[1]
        num_grid_points = np.array(list(len(np.unique(self.grid_points[:, i])) for i in range(g_shape)))

        num_intervals = num_grid_points - 1
        dx = (max_grid - min_grid) / num_intervals

        # Find the real bandwidth, the support times the desired bw factor
        if self.kernel.finite_support:
            real_bw = self.kernel.support * self.bw
        else:
            # The parent class should compute this already. If not, compute
            # it again. This optimization only dominates a little bit with
            # few data points
            try:
                real_bw = self._kernel_practical_support
            except AttributeError:
                real_bw = self.kernel.practical_support(self.bw)

        # Compute L, the number of dx'es to move out from 0 in kernel
        L = np.minimum(np.floor(real_bw / dx), num_intervals + 1)
        assert (dx * L <= real_bw).all()

        # Evaluate the kernel once
        grids = [np.linspace(-dx * L, dx * L, int(L * 2 + 1)) for (dx, L) in zip(dx, L)]
        kernel_grid = cartesian(grids)
        kernel_weights = self.kernel(kernel_grid, bw=self.bw, norm=self.norm)

        # Reshape in preparation to
        kernel_weights = kernel_weights.reshape(*[int(k * 2 + 1) for k in L])
        data = data.reshape(*tuple(num_grid_points))

        # Step 3 - Performing the convolution

        # The following code block surpressed the warning:
        #        anaconda3/lib/python3.6/site-packages/mkl_fft/_numpy_fft.py:
        #            FutureWarning: Using a non-tuple sequence for multidimensional ...
        #        output = mkl_fft.rfftn_numpy(a, s, axes)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            ans = convolve(data, kernel_weights, mode="same").reshape(-1, 1)

        return self._evalate_return_logic(ans, self.grid_points)


## all together

In [ ]:
## writing here 



fig, ax = plt.subplots(1,1, figsize=(14,9))#,\

fs_major=34


for ind_BPS, BPSmodel in enumerate(BPSnameslist[0:2]):

# for ind_BPS, BPSmodel in enumerate(['A']):
    
    print('running model %s'%BPSmodel)
    
    
    # CHIRP MASS 
    print('at mass_1_LVK')
#     print('at chirp mass')
    bw=0.85
    ylim_threshold = 0.0004       
    xparam = 'mass_1_LVK'
#     xparam = 'chirp_mass_LVK'
    plotYlog = False
    xlim, ylim = [0, 48], [0.0, 2.03]

    plotYlog = True
    ylim = [0.0003, 0.15]
    ax = plot_FC_distribution(axe=ax, xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_PDF',\
                                  spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)






xx = np.linspace(1000,20000,100)
yy = np.linspace(1000,20000,100)
lw=12
fs = 30

# ax[2,1].plot(xx, yy,    color=colors_lighter[1], lw=lw, zorder=16, alpha=1, label=r'\textbf{MRR}')#, ls=ls_)
# ax[2,1].plot(xx, yy, color=colors_lighter[0], lw=lw, zorder=15, alpha=1, label=r'\textbf{non-MRR}')#, ls=ls_)
# ax[2,1].legend(fontsize=fs, frameon=False, loc='upper left')

    
plt.tight_layout()  
# plt.subplots_adjust(wspace=0.2, hspace=0.1)



# plt.savefig('./super_FC_split_panel.pdf', transparent=False, bbox_inches="tight",  format='pdf')
# plt.savefig('./super_FC_split_panel.png', transparent=False, bbox_inches="tight", dpi=600, format='png')
plt.show()  











In [ ]:
## writing here 



fig, ax = plt.subplots(1,1, figsize=(14,9))#,\

fs_major=34


for ind_BPS, BPSmodel in enumerate(BPSnameslist[0:]):

# for ind_BPS, BPSmodel in enumerate(['A']):
    
    print('running model %s'%BPSmodel)
    
    
    # CHIRP MASS 
    print('at mass_1_LVK')
#     print('at chirp mass')
    bw=0.85
    ylim_threshold = 0.0004       
    xparam = 'mass_1_LVK'
#     xparam = 'chirp_mass_LVK'
    plotYlog = False
    xlim, ylim = [0, 48], [0.0, 2.03]

    plotYlog = True
    ylim = [0.0003, 0.15]
    ax = plot_FC_distribution(axe=ax, xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_PDF',\
                                  spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)






xx = np.linspace(1000,20000,100)
yy = np.linspace(1000,20000,100)
lw=12
fs = 30

# ax[2,1].plot(xx, yy,    color=colors_lighter[1], lw=lw, zorder=16, alpha=1, label=r'\textbf{MRR}')#, ls=ls_)
# ax[2,1].plot(xx, yy, color=colors_lighter[0], lw=lw, zorder=15, alpha=1, label=r'\textbf{non-MRR}')#, ls=ls_)
# ax[2,1].legend(fontsize=fs, frameon=False, loc='upper left')

    
plt.tight_layout()  
# plt.subplots_adjust(wspace=0.2, hspace=0.1)



plt.savefig('./m1LVK_panel.pdf', transparent=False, bbox_inches="tight",  format='pdf')
plt.savefig('./m1LVK_panel.png', transparent=False, bbox_inches="tight", dpi=600, format='png')
plt.show()  











# 3 PANEL PLOT 

In [ ]:




# fig, ax = plt.subplots(3,3, figsize=(30,8), gridspec_kw={"height_ratios":[1, 1, 2.5], "width_ratios":[1, 1, 1]})#,\

# fs_major=34


# for ind_BPS, BPSmodel in enumerate(BPSnameslist[3:]):

# # for ind_BPS, BPSmodel in enumerate(['A']):
    
#     print('running model %s'%BPSmodel)
    
    
#     # CHIRP MASS 
#     print('at chirp mass')
#     bw=0.85
#     ylim_threshold = 0.0004       
#     xparam = 'chirp_mass_LVK'
#     plotYlog = False
#     xlim, ylim = [0, 58], [0.0, 1.03]

# #     ax[0,0] = plot_FC_distribution(axe=ax[0,0], xparam=xparam, BPSmodelName=BPSmodel, mode='notMRR_fraction',\
# #                                   spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold) 
# #     ax[1,0] = plot_FC_distribution(axe=ax[1,0], xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_fraction',\
# #                                   spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold) 
#     plotYlog = True
#     ylim = [0.0003, 0.15]
#     ax[2,0] = plot_FC_distribution(axe=ax[2,0], xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_PDF',\
#                                   spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)




# #         ## MASS 1##
# #     print('at mass 1 LVK')
# #     xparam = 'mass_1_LVK'
# #     ylim_threshold = 0.0004
# #     bw = 0.85
# #     plotYlog = False
# #     xlim, ylim = [0, 58], [0.0, 1.03]
# # #     ax[0,1] = plot_FC_distribution(axe=ax[0,1], xparam=xparam, BPSmodelName=BPSmodel, mode='notMRR_fraction',\
# # #                                   spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)
# # #     ax[1,1] = plot_FC_distribution(axe=ax[1,1], xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_fraction',\
# # #                                   spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold) 
# #     plotYlog = True
# #     ylim = [0.0003, 0.15]
# #     ax[2,2] = plot_FC_distribution(axe=ax[2,2], xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_PDF',\
# #                                   spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)

        
    

    
    
#     ## MASS RATIO ##
#     print('at mass ratio')
#     xparam = 'mass_ratio_LVK'
#     ylim_threshold = 0.001
#     bw = 0.05
#     plotYlog = False
#     xlim, ylim = [0, 1], [0.0, 1.03]
# #     ax[0,1] = plot_FC_distribution(axe=ax[0,1], xparam=xparam, BPSmodelName=BPSmodel, mode='notMRR_fraction',\
# #                                   spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)
# #     ax[1,1] = plot_FC_distribution(axe=ax[1,1], xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_fraction',\
# #                                   spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold) 
#     plotYlog = True
#     ylim = [0.001, 9.15]
#     ax[2,1] = plot_FC_distribution(axe=ax[2,1], xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_PDF',\
#                                   spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)


    
    



# #     # chi spin  ##
# #     ylim_threshold = 0.001
# #     bw = 0.0051
# #     xparam = 'chi_of_spinning_BH'
# #     plotYlog = False
# #     xlim, ylim = [0.05, 1], [0.0, 1.03] 
# # #     ax[0,2] = plot_FC_distribution(axe=ax[0,2], xparam=xparam, BPSmodelName=BPSmodel, mode='notMRR_fraction',\
# # #                                   spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)
    
# #     ax[1,2] = plot_FC_distribution(axe=ax[1,2], xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_fraction',\
# #                                   spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold) 
# #     plotYlog = False
# #     ylim = [0.5, 1.03]
# #     ax[2,2] = plot_FC_distribution(axe=ax[2,2], xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_CDF',\
# #                                   spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)




# xx = np.linspace(1000,20000,100)
# yy = np.linspace(1000,20000,100)
# lw=12
# fs = 30

# # ax[2,1].plot(xx, yy,    color=colors_lighter[1], lw=lw, zorder=16, alpha=1, label=r'\textbf{MRR}')#, ls=ls_)
# # ax[2,1].plot(xx, yy, color=colors_lighter[0], lw=lw, zorder=15, alpha=1, label=r'\textbf{non-MRR}')#, ls=ls_)
# # ax[2,1].legend(fontsize=fs, frameon=False, loc='upper left')

    
# plt.tight_layout()  
# plt.subplots_adjust(wspace=0.2, hspace=0.1)



# # plt.savefig('./super_FC_split_panel.pdf', transparent=False, bbox_inches="tight",  format='pdf')
# # plt.savefig('./super_FC_split_panel.png', transparent=False, bbox_inches="tight", dpi=600, format='png')
# plt.show()  











In [ ]:
[21.13608489 10.0464932  12.86334076 ...  3.81552635  3.65272609
  3.86739222]
(8371398,)

running model A
at mass_1_LVK
[33.4712  17.0417  14.9915  ...  5.15439  5.54399  5.15051]
(8371398,)
now at channel:  classic

In [ ]:
running model A
at chirp mass
[21.13608489 10.0464932  12.86334076 ...  3.81552635  3.65272609
  3.86739222]
(8371398,)
now at channel:  classic
now at channel:  stable B no CEE
now at channel:  immediate CE
running model B
at chirp mass
[21.59870452  9.69384928  9.8673975  ...  3.16114459  3.28597739
  2.81796725]
(8616008,)
now at channel:  classic
now at channel:  stable B no CEE
now at channel:  immediate CE

In [ ]:
## new 



fig, ax = plt.subplots(1,1, figsize=(14,9))#,\

fs_major=34


for ind_BPS, BPSmodel in enumerate(BPSnameslist[0:]):

# for ind_BPS, BPSmodel in enumerate(['A']):
    
    print('running model %s'%BPSmodel)
    
    
    # CHIRP MASS 
    print('at chirp mass')
    bw=0.85
    ylim_threshold = 0.0004       
    xparam = 'chirp_mass_LVK'
    plotYlog = False
    xlim, ylim = [0, 38], [0.0, 1.03]

    plotYlog = True
    ylim = [0.0003, 0.15]
    ax = plot_FC_distribution(axe=ax, xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_PDF',\
                                  spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)






xx = np.linspace(1000,20000,100)
yy = np.linspace(1000,20000,100)
lw=12
fs = 30

# ax[2,1].plot(xx, yy,    color=colors_lighter[1], lw=lw, zorder=16, alpha=1, label=r'\textbf{MRR}')#, ls=ls_)
# ax[2,1].plot(xx, yy, color=colors_lighter[0], lw=lw, zorder=15, alpha=1, label=r'\textbf{non-MRR}')#, ls=ls_)
# ax[2,1].legend(fontsize=fs, frameon=False, loc='upper left')

    
plt.tight_layout()  
# plt.subplots_adjust(wspace=0.2, hspace=0.1)



# plt.savefig('./super_FC_split_panel.pdf', transparent=False, bbox_inches="tight",  format='pdf')
# plt.savefig('./super_FC_split_panel.png', transparent=False, bbox_inches="tight", dpi=600, format='png')
plt.show()  











In [ ]:




fig, ax = plt.subplots(1,1, figsize=(14,10))#,\

fs_major=34


for ind_BPS, BPSmodel in enumerate(BPSnameslist[0:2]):

# for ind_BPS, BPSmodel in enumerate(['A']):
    
    print('running model %s'%BPSmodel)
    
    
        ## MASS 1##
    print('at mass 1 LVK')
    xparam = 'mass_1_LVK'
    ylim_threshold = 0.0004
    bw = 0.85
    plotYlog = False
    xlim, ylim = [0, 40], [0.0, 1.03]
#     ax[0,1] = plot_FC_distribution(axe=ax[0,1], xparam=xparam, BPSmodelName=BPSmodel, mode='notMRR_fraction',\
#                                   spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)
#     ax[1,1] = plot_FC_distribution(axe=ax[1,1], xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_fraction',\
#                                   spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold) 
    plotYlog = True
    ylim = [0.0003, 0.15]

    ax = plot_FC_distribution(axe=ax, xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_PDF',\
                                  spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)






xx = np.linspace(1000,20000,100)
yy = np.linspace(1000,20000,100)
lw=12
fs = 30

# ax[2,1].plot(xx, yy,    color=colors_lighter[1], lw=lw, zorder=16, alpha=1, label=r'\textbf{MRR}')#, ls=ls_)
# ax[2,1].plot(xx, yy, color=colors_lighter[0], lw=lw, zorder=15, alpha=1, label=r'\textbf{non-MRR}')#, ls=ls_)
# ax[2,1].legend(fontsize=fs, frameon=False, loc='upper left')

    
plt.tight_layout()  
# plt.subplots_adjust(wspace=0.2, hspace=0.1)



# plt.savefig('./super_FC_split_panel.pdf', transparent=False, bbox_inches="tight",  format='pdf')
# plt.savefig('./super_FC_split_panel.png', transparent=False, bbox_inches="tight", dpi=600, format='png')
plt.show()  











In [ ]:




fig, ax = plt.subplots(1,1, figsize=(14,10))#,\

fs_major=34


for ind_BPS, BPSmodel in enumerate(BPSnameslist[0:2]):

# for ind_BPS, BPSmodel in enumerate(['A']):
    
    print('running model %s'%BPSmodel)
    
    
    # CHIRP MASS 
    print('at chirp mass')
    bw=0.85
    ylim_threshold = 0.0004       
    xparam = 'chirp_mass_LVK'
    plotYlog = False
    xlim, ylim = [0, 38], [0.0, 1.03]

    plotYlog = True
    ylim = [0.0003, 0.15]
    ax = plot_FC_distribution(axe=ax, xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_PDF',\
                                  spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)






xx = np.linspace(1000,20000,100)
yy = np.linspace(1000,20000,100)
lw=12
fs = 30

# ax[2,1].plot(xx, yy,    color=colors_lighter[1], lw=lw, zorder=16, alpha=1, label=r'\textbf{MRR}')#, ls=ls_)
# ax[2,1].plot(xx, yy, color=colors_lighter[0], lw=lw, zorder=15, alpha=1, label=r'\textbf{non-MRR}')#, ls=ls_)
# ax[2,1].legend(fontsize=fs, frameon=False, loc='upper left')

    
plt.tight_layout()  
# plt.subplots_adjust(wspace=0.2, hspace=0.1)



# plt.savefig('./super_FC_split_panel.pdf', transparent=False, bbox_inches="tight",  format='pdf')
# plt.savefig('./super_FC_split_panel.png', transparent=False, bbox_inches="tight", dpi=600, format='png')
plt.show()  











In [ ]:




fig, ax = plt.subplots(3,3, figsize=(30,8), gridspec_kw={"height_ratios":[1, 1, 2.5], "width_ratios":[1, 1, 1]})#,\

fs_major=34


for ind_BPS, BPSmodel in enumerate(BPSnameslist[:2]):

    
    print('running model %s'%BPSmodel)

        ## MASS 1##
    print('at mass 1 LVK')
    xparam = 'chirp_mass_LVK' #'mass_1_LVK'
    ylim_threshold = 0.0004
    bw = 0.85
    plotYlog = False
    xlim, ylim = [0, 45], [0.0, 1.03]
#     ax[0,1] = plot_FC_distribution(axe=ax[0,1], xparam=xparam, BPSmodelName=BPSmodel, mode='notMRR_fraction',\
#                                   spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)
#     ax[1,1] = plot_FC_distribution(axe=ax[1,1], xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_fraction',\
#                                   spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold) 
    plotYlog = True
    ylim = [0.0003, 0.15]
    ax[2,0] = plot_FC_distribution(axe=ax[2,0], xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_PDF',\
                                  spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)

        

xx = np.linspace(1000,20000,100)
yy = np.linspace(1000,20000,100)
lw=12
fs = 30


plt.tight_layout()  
plt.subplots_adjust(wspace=0.2, hspace=0.1)

# plt.savefig('./single_FC_split_panel.pdf', transparent=False, bbox_inches="tight",  format='pdf')
# plt.savefig('./single_FC_split_panel.png', transparent=False, bbox_inches="tight", dpi=600, format='png')
plt.show()  











In [ ]:




# def plot_FC_distribution(axe='None', xparam='chiEff', BPSmodelName='A', mode='pdf',\
#                           spin_threshold='None', bw=0.01, xlim=[0,1], ylim=[0,1],\
#                           plotYlog='False', ylim_threshold=0.02):#, mssfr='112'):
    
#     fs_l = 28 # label fontsize
    
# #     colors = ['#1f77b4', '#ff7f0e']
# #     colors_lighter = [adjust_lightness(color=colors[0], amount=2.2),adjust_lightness(color=colors[1], amount=1.7)]

#     DCOtype='BBH'
#     # path for files 
#     path_ = '/Volumes/Andromeda2/DATA/AllDCO_bugfix/' + alphabetDirDict[BPSmodelName] +'/'
#     path  = path_ + 'COMPASCompactOutput_'+ DCOtype +'_' + BPSmodelName + '.h5'
    
#     fdata = h5.File(path, 'r')
#     massCO_ZAMSM1 = fdata['doubleCompactObjects']['M1'][...].squeeze()
#     massCO_ZAMSM2 = fdata['doubleCompactObjects']['M2'][...].squeeze()
#     # M1 will be the most massive, M2 the least massive compact object. 
#     massCO_LVKM1, massCO_LVKM2 = obtainM1BHandM2BHassymetric(m1=fdata['doubleCompactObjects']['M1'][...].squeeze(), m2=fdata['doubleCompactObjects']['M2'][...].squeeze()) 
#     MassRatioCO_LVK = massCO_LVKM2/massCO_LVKM1
    
#     seedsDCO  = fdata['doubleCompactObjects']['seed'][...].squeeze()  # get the seeds in the DCO file 
#     seedsSN   = fdata['supernovae']['randomSeed'][...].squeeze()    # get the seeds in the SN file 
#     indices   = np.sort(np.unique(seedsSN[1::2], return_index=True)[1])
#     maskSNdco = np.in1d(seedsSN,  seedsDCO) # mask in the SNe files the SNe that correspond to our DCO
#     whichSN   = fdata['supernovae']['whichStar'][...].squeeze()[maskSNdco]  # this is 1 if the initially primary star goes SN and 2 if the secondary goes supernova     
#     whichSN2  = whichSN[1::2][indices]

#     # either SN2 = primary (1) and M1 is > M2, or SN2 = secondary & M1 < M2 
#     # this takes into account (first term) rejuvenation 
#     mask_MRR = ((whichSN2==1) & (massCO_ZAMSM1>massCO_ZAMSM2) ) | ((whichSN2==2) & (massCO_ZAMSM1<massCO_ZAMSM2)) 

    
    
#     # obtain formation channels 
#     seeds = fdata['doubleCompactObjects']['seed'][...].squeeze()
#     channels = identify_formation_channels(seeds=seeds, file=fdata)
    
    
    
#     del massCO_ZAMSM1
#     del massCO_ZAMSM2
#     del whichSN2
#     del whichSN
#     del maskSNdco
#     del indices
#     del seedsSN
#     del seedsDCO


    
#     if (mode in ['spin_PDF', 'spin_fraction', 'spinOne_fraction', 'spinTwo_fraction', 'm1spin_or_m2spin_fraction', 'spin_CDF' ]) | (xparam in ['chi_of_spinning_BH', 'chi_effective', 'log10_t_delay']):
#         spin = COspin(data_path=path, state='he_depletion')  # set class 
#         spin.setCOMPASData() # reads in the COMPAS DCO parameters 
#         spinZAMSM1, spinZAMSM2  = spin.BaveraSpin() #ZAMS M1 SPIN 

#         spinLVKM1, spinLVKM2 = np.zeros_like(spinZAMSM1), np.zeros_like(spinZAMSM1)
#         spinLVKM1[mask_MRR] = spinZAMSM2[mask_MRR]  # MRR so M1 comes from M2ZAMS, we assign it spin from M2ZAMS
#         spinLVKM1[~mask_MRR] = spinZAMSM1[~mask_MRR]  # no MRR so M1 comes from M1ZAMS, we assign it spin from M1ZAMS
#         spinLVKM2[mask_MRR] = spinZAMSM1[mask_MRR]   # MRR so M2 comes from M1ZAMS, we assign it spin from M1ZAMS
#         spinLVKM2[~mask_MRR] = spinZAMSM2[~mask_MRR]   # no MRR so M2 comes from M2ZAMS, we assign it spin from M2ZAMS     

#         # spin_threshold = 0.05 # definition of "spinning BH"
#         mask_LVKM1_spinning = (spinLVKM1 > spin_threshold ) 
#         mask_LVKM2_spinning = (spinLVKM2 > spin_threshold ) # definition of "spinning BH"
#         mask_anySpin = (spinLVKM1 > spin_threshold ) | (spinLVKM2 > spin_threshold )    

    

#     if xparam=='chirp_mass_LVK':
#         param_x = chirpmass(massCO_LVKM1, massCO_LVKM2)
#         nameX = r'$\mathcal{M}_{\rm{c}} \ [M_{\odot}]$'
#         nameY = r'\textbf{PDF}'
#         xx = np.linspace(1,100,1000)
#         print(param_x)
#         print(np.shape(param_x))
        
#     elif xparam=='mass_ratio_LVK':
#         param_x = MassRatioCO_LVK
#         nameX = r'$q$'
#         nameY = r'\textbf{PDF}'
#         xx = np.linspace(-0.2,1.2,1000)

        

#     elif xparam=='mass_1_LVK':
#         param_x = massCO_LVKM1
#         nameX = r'$m_1$'
#         nameY = r'\textbf{PDF}'
#         xx = np.linspace(1,100,1000)
#         print(param_x)
#         print(np.shape(param_x))
        
#     elif xparam=='chi_of_spinning_BH':
#         param_x = spinLVKM1 + spinLVKM2

#         nameX = r'$\chi_{\rm{i}}$'
#         nameY = r'\textbf{PDF}'
#         xx = np.linspace(-0.2,2,1000)  

        
#     elif xparam=='chi_effective':
#         param_x = ((spinLVKM1*massCO_LVKM1) + (spinLVKM2*massCO_LVKM2)) / (massCO_LVKM1+massCO_LVKM2)
#         nameX = r'$\chi_{\rm{eff}}$'
#         nameY = r'\textbf{PDF}'
#         xx = np.linspace(-0.2,2,1000)          
        
#     elif xparam=='log10_t_delay':
#         param_x = (fdata['doubleCompactObjects']['tform'][...].squeeze() +  fdata['doubleCompactObjects']['tc'][...].squeeze() ) / 1000 # divide by 1000 to make it in [Gyr]
#         param_x = np.log10(param_x)
#         nameX = r'$t_{\rm{delay}} \ [\rm{Gyr}]$'
#         nameY = r'\textbf{PDF}'  
#         xx = np.linspace(-4,2,1000)  

    
#     if (mode=='MRR_PDF') | (mode=='spin_PDF'):
#         nameY = r'\textbf{PDF}'
#     elif (mode=='MRR_fraction'):
#         nameY = r'$\rm{f}_{\rm{channel}}$'
#     elif (mode=='notMRR_fraction'):
#         nameY = r'$\rm{f}_{\rm{non MRR}}$'
#     elif (mode=='spin_fraction'):
#         nameY =   r'$\rm{f}_{\chi_{1} > %s}$'%spin_threshold
#     elif (mode=='m1spin_or_m2spin_fraction'):
#         nameY =   r'$\rm{f}_{\chi_{\rm{i}} > %s}$'%spin_threshold 
#     elif (mode=='MRR_CDF') | (mode=='spin_CDF'):
#         nameY = r'\textbf{CDF}'
#     elif (mode=='spinOne_fraction'):
#         nameY =   r'$\rm{f}_{\chi_{1} > %s}$'%spin_threshold
#     elif (mode=='spinTwo_fraction'):
#         nameY =   r'$\rm{f}_{\chi_{2} > %s}$'%spin_threshold
#     else:
#         raise ValueError("the provided `mode` is not recognized, the value given is %s"%mode)
        
    
#     estimator = FFTKDE(kernel='biweight', bw=bw)
    
#     if plotYlog==True:       
#         axe.set_yscale('log')
        
        
        
# #     channelList = ['classic', 'stable B no CEE', 'vii',  'immediate CE',  r'double-core CE', 'other']
#     adjustedChannelList = ['classic', 'stable B no CEE', 'immediate CE']#,  r'double-core CE']
# #     adjustedChannelList = ['classic']#, 'stable B no CEE', 'immediate CE']#
#     for nrC, Channel in enumerate(adjustedChannelList): 
        
#         print('now at channel: ', Channel)
#         ind_wanted = dictFormationChannelIndex[Channel]
        
        
        
#         c_FC = channelColorDict[Channel]
#         colors_lighter_FC =  channelColorDict_lighter[Channel]
# #         print('now at channel', Channel)




#         mask_MRR = (channels==ind_wanted)
    
#         if len(mask_MRR)>0:
    
#     #         for ind_mssfr, mssfr in enumerate(MSSFRnameslist[0:]):
#             for ind_mssfr, mssfr in enumerate(MSSFRnameslist[0:2]):    

#                 ls_ = '-'
#                 Highlight = False
#                 if ((mssfr in ['123']) & (BPSmodelName in ['K'])):
#                     ls_ = ':'
#                     Highlight = True
#                 elif  ((mssfr in ['312']) & (BPSmodelName in ['T'])):
#                     ls_ = '--'
#                     Highlight = True
#                 elif ((mssfr in ['231']) & (BPSmodelName in ['O'])):
#                     ls_ = '-.'
#                     Highlight = True




#                 ### read in MSSFR weights: ###
#     #             fparam_key = 'weights_detected'
#                 fparam_key = 'weights_intrinsic'
#                 weightheader = 'w_' + mssfr
#                 weights_ = fdata[fparam_key][weightheader][...].squeeze()
#                 w = weights_
                
# #                 print('weights', w)
# #                 print(np.isnan(w).any())
# #                 print(np.isinf(w).any())
# #                 w = np.nan_to_num(w)
# #                 w = np.inf_to_num(w)
                
                

#                 if (mode=='MRR_PDF') | (mode=='MRR_fraction') |  (mode=='notMRR_fraction') : 
#     #                 yy_nonMRR = estimator.fit(param_x[~mask_MRR], weights=w[~mask_MRR]).evaluate(xx)
#                     yy_MRR = estimator.fit(param_x[mask_MRR], weights=w[mask_MRR]).evaluate(xx) 
#                     yy_total = estimator.fit(param_x, weights=w).evaluate(xx) 

#     #                 rel_weight_nonMRR = np.sum(w[~mask_MRR]) / (np.sum(w))
#                     rel_weight_MRR    = np.sum(w[mask_MRR])  / (np.sum(w))
#     #                 yy_nonMRR *= rel_weight_nonMRR
#                     yy_MRR *= rel_weight_MRR


#                 elif mode in ['spin_PDF', 'spin_fraction', 'spinOne_fraction', 'spinTwo_fraction']:
#     #                 yy_nonMRR = estimator.fit(param_x[mask_LVKM2_spinning], weights=w[mask_LVKM2_spinning]).evaluate(xx)
#                     yy_MRR = estimator.fit(param_x[mask_LVKM1_spinning], weights=w[mask_LVKM1_spinning]).evaluate(xx)
#                     yy_total = estimator.fit(param_x, weights=w).evaluate(xx)

#     #                 rel_weight_nonMRR = np.sum(w[mask_LVKM2_spinning]) / (np.sum(w))
#                     rel_weight_MRR    = np.sum(w[mask_LVKM1_spinning])  / (np.sum(w))
#     #                 yy_nonMRR *= rel_weight_nonMRR
#                     yy_MRR *= rel_weight_MRR


#                 ## IF WE ARE PLOTTING CDFs
#                 elif (mode=='MRR_CDF') | (mode=='spin_CDF'):
#                     if Highlight==True:
#     #                     axe.hist(param_x[~mask_MRR], bins=10000, color=colors[0], lw=3, zorder=15, alpha=1, ls=ls_ , density=True, weights=weights_[~mask_MRR], cumulative=True, histtype='step')
#                         axe.hist(param_x[mask_MRR],  bins=10000, color= c_FC, lw=3, zorder=16, alpha=1, ls=ls_,   density=True, weights=weights_[mask_MRR],  cumulative=True, histtype='step')
#                     else:
#     #                     axe.hist(param_x[~mask_MRR], bins=10000, color=colors_lighter[0], lw=1.5, zorder=5, alpha=1, ls=ls_, density=True, weights=weights_[~mask_MRR], cumulative=True, histtype='step')
#                         axe.hist(param_x[mask_MRR],  bins=10000, color=colors_lighter_FC, lw=1.5, zorder=6, alpha=1, ls=ls_,  density=True, weights=weights_[mask_MRR],  cumulative=True, histtype='step')

#                     axe = layoutAxes(axe, nameX=nameX, nameY=nameY, setMinor=False, labelpad=5, fontsize=fs_l, labelSizeMajor=fs_major)                        


#                 ### PLOTTING ###
#                 ## IF WE ARE PLOTTING PDFs
#                 if (mode=='MRR_PDF') | (mode=='spin_PDF'):
#                     if Highlight==True:
#     #                     axe.plot(xx, yy_nonMRR, color=colors[0], lw=3, zorder=15, alpha=1, ls=ls_)
#                         axe.plot(xx, yy_MRR,    color=c_FC, lw=3, zorder=16, alpha=1, ls=ls_)
#                     else:
#     #                     axe.plot(xx, yy_nonMRR, color=colors_lighter[0], lw=1.5, zorder=5, alpha=1, ls=ls_)
#                         axe.plot(xx, yy_MRR,    color=colors_lighter_FC, lw=3, zorder=6, alpha=1, ls=ls_)

#                     axe = layoutAxes(axe, nameX=nameX, nameY=nameY, setMinor=False, labelpad=0, fontsize=fs_l, labelSizeMajor=fs_major)  


#                 # PLOTTING FRACTION     
#                 elif mode in ['MRR_fraction', 'spin_fraction',  'notMRR_fraction', 'spinOne_fraction', 'spinTwo_fraction']: 

#     #                 yy_total = (yy_nonMRR + yy_MRR)
#                     # in some cases the overall PDF is so small, that the ratio is dominated by some shot noise. So we make those zero. 
#                     mask_too_small_values =  (yy_MRR < ylim_threshold)
#                     yy_MRR[mask_too_small_values] = np.zeros(int(np.sum(mask_too_small_values)))
#     #                 yy_nonMRR[mask_too_small_values] = np.zeros(int(np.sum(mask_too_small_values)))

#                     if (mode=='MRR_fraction') | (mode=='spin_fraction') | (mode=='spinOne_fraction'):
#                         ratio = yy_MRR/yy_total

#     #                 elif (mode=='notMRR_fraction') | (mode=='spinTwo_fraction'): 
#     #                     ratio = yy_nonMRR/yy_total
#     #                     color = colors[0]
#     #                     color_lighter = colors_lighter[0]

#                     if Highlight==True:
#                         axe.plot(xx, ratio,     color=c_FC,          lw=3,  zorder=20, alpha=1, ls=ls_)
#                     else:
#                         axe.plot(xx, ratio,     color=colors_lighter_FC,  lw=1.5, zorder=6, alpha=1, ls=ls_)

#                     # remove labels and ticks  on X axe
#                     axe = layoutAxesNoXlabel(axe, nameX=nameX, nameY=nameY, setMinor=False, labelpad=5, fontsize=fs_l, labelSizeMajor=fs_major)
#                     for tick in axe.xaxis.get_major_ticks():
#                         tick.tick1line.set_visible(False)
#                         tick.tick2line.set_visible(False)
#                         tick.label1.set_visible(False)
#                         tick.label2.set_visible(False)


#                 elif (mode=='m1spin_or_m2spin_fraction'):

#                     yy_total = estimator.fit(param_x, weights=w).evaluate(xx)
#                     yy_anySpin = estimator.fit(param_x[mask_anySpin], weights=w[mask_anySpin]).evaluate(xx)


#                     # in some cases the overall PDF is so small, that the ratio is dominated by some shot noise. So we make those zero. 
#                     mask_too_small_values = (yy_anySpin < ylim_threshold) & (yy_total < ylim_threshold)
#                     yy_anySpin[mask_too_small_values] = np.zeros(int(np.sum(mask_too_small_values)))            


#                     if Highlight==True:
#                         axe.plot(xx, yy_anySpin*np.sum(w[mask_anySpin]) / (yy_total*np.sum(w)),     color='k',        lw=2, zorder=20, alpha=1, ls=ls_)
#                     else:
#                         axe.plot(xx, yy_anySpin*np.sum(w[mask_anySpin]) / (yy_total*np.sum(w)),     color='lightgray',      lw=2, zorder=6, alpha=1, ls=ls_)


#                     # remove labels and ticks  on X axe
#                     axe = layoutAxesNoXlabel(axe, nameX=nameX, nameY=nameY, setMinor=False, labelpad=5, fontsize=fs_l, labelSizeMajor=fs_major)
#                     for tick in axe.xaxis.get_major_ticks():
#                         tick.tick1line.set_visible(False)
#                         tick.tick2line.set_visible(False)
#                         tick.label1.set_visible(False)
#                         tick.label2.set_visible(False)

#     if mode in [ 'MRR_fraction', 'notMRR_fraction', 'spinTwo_fraction', 'spinOne_fraction', 'm1spin_or_m2spin_fraction']:
#         axe.set_yticks([ 0.5, 1])
#     elif mode in ['MRR_CDF', 'spin_CDF']:
#         axe.set_yticks([0.5, 0.75, 1])
        

            
#     axe.set_xlim(xlim[0], xlim[1])
#     axe.set_ylim(ylim[0], ylim[1])
#     axe.grid(True)
    
    


    
#     return axe 









# OLD CODE 

In [ ]:


channelColorDict = {1:'#118AB2', 2:'#EF476F', 3:'#FFD166', 4:'#073B4C', 0:'gray'}

channelColorDict_lighter = {1:adjust_lightness(color='#118AB2', amount=1.6), 2:adjust_lightness(color='#EF476F', amount=1.4), 3:adjust_lightness(color='#FFD166', amount=1.2), 4:adjust_lightness(color='#073B4C', amount=1.8), 0:adjust_lightness(color='gray', amount=1.5)}


def plot_FC_distribution(axe='None', xparam='chiEff', BPSmodelName='A', mode='pdf',\
                          spin_threshold='None', bw=0.01, xlim=[0,1], ylim=[0,1],\
                          plotYlog='False', ylim_threshold=0.02):#, mssfr='112'):
    
    fs_l = 28 # label fontsize
    
    colors = ['#1f77b4', '#ff7f0e']
    colors_lighter = [adjust_lightness(color=colors[0], amount=2.2),adjust_lightness(color=colors[1], amount=1.7)]

    DCOtype='BBH'
    # path for files 
    path_ = '/Volumes/Andromeda2/DATA/AllDCO_bugfix/' + alphabetDirDict[BPSmodelName] +'/'
    path  = path_ + 'COMPASCompactOutput_'+ DCOtype +'_' + BPSmodelName + '.h5'
    
    fdata = h5.File(path, 'r')
    massCO_ZAMSM1 = fdata['doubleCompactObjects']['M1'][...].squeeze()
    massCO_ZAMSM2 = fdata['doubleCompactObjects']['M2'][...].squeeze()
    # M1 will be the most massive, M2 the least massive compact object. 
    massCO_LVKM1, massCO_LVKM2 = obtainM1BHandM2BHassymetric(m1=fdata['doubleCompactObjects']['M1'][...].squeeze(), m2=fdata['doubleCompactObjects']['M2'][...].squeeze()) 
    MassRatioCO_LVK = massCO_LVKM2/massCO_LVKM1
    
    seedsDCO  = fdata['doubleCompactObjects']['seed'][...].squeeze()  # get the seeds in the DCO file 
    seedsSN   = fdata['supernovae']['randomSeed'][...].squeeze()    # get the seeds in the SN file 
    indices   = np.sort(np.unique(seedsSN[1::2], return_index=True)[1])
    maskSNdco = np.in1d(seedsSN,  seedsDCO) # mask in the SNe files the SNe that correspond to our DCO
    whichSN   = fdata['supernovae']['whichStar'][...].squeeze()[maskSNdco]  # this is 1 if the initially primary star goes SN and 2 if the secondary goes supernova     
    whichSN2  = whichSN[1::2][indices]

    # either SN2 = primary (1) and M1 is > M2, or SN2 = secondary & M1 < M2 
    # this takes into account (first term) rejuvenation 
    mask_MRR = ((whichSN2==1) & (massCO_ZAMSM1>massCO_ZAMSM2) ) | ((whichSN2==2) & (massCO_ZAMSM1<massCO_ZAMSM2)) 

    
    
    # obtain formation channels 
    seeds = fdata['doubleCompactObjects']['seed'][...].squeeze()
    channels = identify_formation_channels(seeds=seeds, file=fdata)
    
    
    
    del massCO_ZAMSM1
    del massCO_ZAMSM2
    del whichSN2
    del whichSN
    del maskSNdco
    del indices
    del seedsSN
    del seedsDCO


    
    if (mode in ['spin_PDF', 'spin_fraction', 'spinOne_fraction', 'spinTwo_fraction', 'm1spin_or_m2spin_fraction', 'spin_CDF' ]) | (xparam in ['chi_of_spinning_BH', 'chi_effective', 'log10_t_delay']):
        spin = COspin(data_path=path, state='he_depletion')  # set class 
        spin.setCOMPASData() # reads in the COMPAS DCO parameters 
        spinZAMSM1, spinZAMSM2  = spin.BaveraSpin() #ZAMS M1 SPIN 

        spinLVKM1, spinLVKM2 = np.zeros_like(spinZAMSM1), np.zeros_like(spinZAMSM1)
        spinLVKM1[mask_MRR] = spinZAMSM2[mask_MRR]  # MRR so M1 comes from M2ZAMS, we assign it spin from M2ZAMS
        spinLVKM1[~mask_MRR] = spinZAMSM1[~mask_MRR]  # no MRR so M1 comes from M1ZAMS, we assign it spin from M1ZAMS
        spinLVKM2[mask_MRR] = spinZAMSM1[mask_MRR]   # MRR so M2 comes from M1ZAMS, we assign it spin from M1ZAMS
        spinLVKM2[~mask_MRR] = spinZAMSM2[~mask_MRR]   # no MRR so M2 comes from M2ZAMS, we assign it spin from M2ZAMS     

        # spin_threshold = 0.05 # definition of "spinning BH"
        mask_LVKM1_spinning = (spinLVKM1 > spin_threshold ) 
        mask_LVKM2_spinning = (spinLVKM2 > spin_threshold ) # definition of "spinning BH"
        mask_anySpin = (spinLVKM1 > spin_threshold ) | (spinLVKM2 > spin_threshold )    

    

    if xparam=='chirp_mass_LVK':
        param_x = chirpmass(massCO_LVKM1, massCO_LVKM2)
        nameX = r'$\mathcal{M}_{\rm{c}} \ [M_{\odot}]$'
        nameY = r'\textbf{PDF}'
        xx = np.linspace(1,100,1000)

    elif xparam=='mass_ratio_LVK':
        param_x = MassRatioCO_LVK
        nameX = r'$q$'
        nameY = r'\textbf{PDF}'
        xx = np.linspace(-0.2,1.2,1000)
        
    elif xparam=='chi_of_spinning_BH':
        param_x = spinLVKM1 + spinLVKM2

        nameX = r'$\chi_{\rm{i}}$'
        nameY = r'\textbf{PDF}'
        xx = np.linspace(-0.2,2,1000)  

        
    elif xparam=='chi_effective':
        param_x = ((spinLVKM1*massCO_LVKM1) + (spinLVKM2*massCO_LVKM2)) / (massCO_LVKM1+massCO_LVKM2)
        nameX = r'$\chi_{\rm{eff}}$'
        nameY = r'\textbf{PDF}'
        xx = np.linspace(-0.2,2,1000)          
        
    elif xparam=='log10_t_delay':
        param_x = (fdata['doubleCompactObjects']['tform'][...].squeeze() +  fdata['doubleCompactObjects']['tc'][...].squeeze() ) / 1000 # divide by 1000 to make it in [Gyr]
        param_x = np.log10(param_x)
        nameX = r'$t_{\rm{delay}} \ [\rm{Gyr}]$'
        nameY = r'\textbf{PDF}'  
        xx = np.linspace(-4,2,1000)  

    
    if (mode=='MRR_PDF') | (mode=='spin_PDF'):
        nameY = r'\textbf{PDF}'
    elif (mode=='MRR_fraction'):
        nameY = r'$\rm{f}_{\rm{channel}}$'
    elif (mode=='notMRR_fraction'):
        nameY = r'$\rm{f}_{\rm{non MRR}}$'
    elif (mode=='spin_fraction'):
        nameY =   r'$\rm{f}_{\chi_{1} > %s}$'%spin_threshold
    elif (mode=='m1spin_or_m2spin_fraction'):
        nameY =   r'$\rm{f}_{\chi_{\rm{i}} > %s}$'%spin_threshold 
    elif (mode=='MRR_CDF') | (mode=='spin_CDF'):
        nameY = r'\textbf{CDF}'
    elif (mode=='spinOne_fraction'):
        nameY =   r'$\rm{f}_{\chi_{1} > %s}$'%spin_threshold
    elif (mode=='spinTwo_fraction'):
        nameY =   r'$\rm{f}_{\chi_{2} > %s}$'%spin_threshold
    else:
        raise ValueError("the provided `mode` is not recognized, the value given is %s"%mode)
        
    
    estimator = FFTKDE(kernel='biweight', bw=bw)
    
    if plotYlog==True:       
        axe.set_yscale('log')
        
        
        

    for nrC, Channel in enumerate(list(np.unique(channels))): 
        
        
        mask_MRR = (channels==Channel)
        c_FC = channelColorDict[Channel]
        colors_lighter_FC =  channelColorDict_lighter[Channel]
#         print('now at channel', Channel)
    
#         for ind_mssfr, mssfr in enumerate(MSSFRnameslist[0:]):
        for ind_mssfr, mssfr in enumerate(MSSFRnameslist[0:]):    

            ls_ = '-'
            Highlight = False
            if ((mssfr in ['123']) & (BPSmodelName in ['K'])):
                ls_ = ':'
                Highlight = True
            elif  ((mssfr in ['312']) & (BPSmodelName in ['T'])):
                ls_ = '--'
                Highlight = True
            elif ((mssfr in ['231']) & (BPSmodelName in ['O'])):
                ls_ = '-.'
                Highlight = True




            ### read in MSSFR weights: ###
            fparam_key = 'weights_detected'
            weightheader = 'w_' + mssfr
            weights_ = fdata[fparam_key][weightheader][...].squeeze()
            w = weights_

            if (mode=='MRR_PDF') | (mode=='MRR_fraction') |  (mode=='notMRR_fraction') : 
#                 yy_nonMRR = estimator.fit(param_x[~mask_MRR], weights=w[~mask_MRR]).evaluate(xx)
                yy_MRR = estimator.fit(param_x[mask_MRR], weights=w[mask_MRR]).evaluate(xx) 
                yy_total = estimator.fit(param_x, weights=w).evaluate(xx) 
        
#                 rel_weight_nonMRR = np.sum(w[~mask_MRR]) / (np.sum(w))
                rel_weight_MRR    = np.sum(w[mask_MRR])  / (np.sum(w))
#                 yy_nonMRR *= rel_weight_nonMRR
                yy_MRR *= rel_weight_MRR


            elif mode in ['spin_PDF', 'spin_fraction', 'spinOne_fraction', 'spinTwo_fraction']:
#                 yy_nonMRR = estimator.fit(param_x[mask_LVKM2_spinning], weights=w[mask_LVKM2_spinning]).evaluate(xx)
                yy_MRR = estimator.fit(param_x[mask_LVKM1_spinning], weights=w[mask_LVKM1_spinning]).evaluate(xx)
                yy_total = estimator.fit(param_x, weights=w).evaluate(xx)
        
#                 rel_weight_nonMRR = np.sum(w[mask_LVKM2_spinning]) / (np.sum(w))
                rel_weight_MRR    = np.sum(w[mask_LVKM1_spinning])  / (np.sum(w))
#                 yy_nonMRR *= rel_weight_nonMRR
                yy_MRR *= rel_weight_MRR


            ## IF WE ARE PLOTTING CDFs
            elif (mode=='MRR_CDF') | (mode=='spin_CDF'):
                if Highlight==True:
#                     axe.hist(param_x[~mask_MRR], bins=10000, color=colors[0], lw=3, zorder=15, alpha=1, ls=ls_ , density=True, weights=weights_[~mask_MRR], cumulative=True, histtype='step')
                    axe.hist(param_x[mask_MRR],  bins=10000, color= c_FC, lw=3, zorder=16, alpha=1, ls=ls_,   density=True, weights=weights_[mask_MRR],  cumulative=True, histtype='step')
                else:
#                     axe.hist(param_x[~mask_MRR], bins=10000, color=colors_lighter[0], lw=1.5, zorder=5, alpha=1, ls=ls_, density=True, weights=weights_[~mask_MRR], cumulative=True, histtype='step')
                    axe.hist(param_x[mask_MRR],  bins=10000, color=colors_lighter_FC, lw=1.5, zorder=6, alpha=1, ls=ls_,  density=True, weights=weights_[mask_MRR],  cumulative=True, histtype='step')

                axe = layoutAxes(axe, nameX=nameX, nameY=nameY, setMinor=False, labelpad=5, fontsize=fs_l, labelSizeMajor=fs_major)                        


            ### PLOTTING ###
            ## IF WE ARE PLOTTING PDFs
            if (mode=='MRR_PDF') | (mode=='spin_PDF'):
                if Highlight==True:
#                     axe.plot(xx, yy_nonMRR, color=colors[0], lw=3, zorder=15, alpha=1, ls=ls_)
                    axe.plot(xx, yy_MRR,    color=c_FC, lw=3, zorder=16, alpha=1, ls=ls_)
                else:
#                     axe.plot(xx, yy_nonMRR, color=colors_lighter[0], lw=1.5, zorder=5, alpha=1, ls=ls_)
                    axe.plot(xx, yy_MRR,    color=colors_lighter_FC, lw=1.5, zorder=6, alpha=1, ls=ls_)

                axe = layoutAxes(axe, nameX=nameX, nameY=nameY, setMinor=False, labelpad=0, fontsize=fs_l, labelSizeMajor=fs_major)  


            # PLOTTING FRACTION     
            elif mode in ['MRR_fraction', 'spin_fraction',  'notMRR_fraction', 'spinOne_fraction', 'spinTwo_fraction']: 

#                 yy_total = (yy_nonMRR + yy_MRR)
                # in some cases the overall PDF is so small, that the ratio is dominated by some shot noise. So we make those zero. 
                mask_too_small_values =  (yy_MRR < ylim_threshold)
                yy_MRR[mask_too_small_values] = np.zeros(int(np.sum(mask_too_small_values)))
#                 yy_nonMRR[mask_too_small_values] = np.zeros(int(np.sum(mask_too_small_values)))

                if (mode=='MRR_fraction') | (mode=='spin_fraction') | (mode=='spinOne_fraction'):
                    ratio = yy_MRR/yy_total

#                 elif (mode=='notMRR_fraction') | (mode=='spinTwo_fraction'): 
#                     ratio = yy_nonMRR/yy_total
#                     color = colors[0]
#                     color_lighter = colors_lighter[0]

                if Highlight==True:
                    axe.plot(xx, ratio,     color=c_FC,          lw=3,  zorder=20, alpha=1, ls=ls_)
                else:
                    axe.plot(xx, ratio,     color=colors_lighter_FC,  lw=1.5, zorder=6, alpha=1, ls=ls_)

                # remove labels and ticks  on X axe
                axe = layoutAxesNoXlabel(axe, nameX=nameX, nameY=nameY, setMinor=False, labelpad=5, fontsize=fs_l, labelSizeMajor=fs_major)
                for tick in axe.xaxis.get_major_ticks():
                    tick.tick1line.set_visible(False)
                    tick.tick2line.set_visible(False)
                    tick.label1.set_visible(False)
                    tick.label2.set_visible(False)


            elif (mode=='m1spin_or_m2spin_fraction'):

                yy_total = estimator.fit(param_x, weights=w).evaluate(xx)
                yy_anySpin = estimator.fit(param_x[mask_anySpin], weights=w[mask_anySpin]).evaluate(xx)


                # in some cases the overall PDF is so small, that the ratio is dominated by some shot noise. So we make those zero. 
                mask_too_small_values = (yy_anySpin < ylim_threshold) & (yy_total < ylim_threshold)
                yy_anySpin[mask_too_small_values] = np.zeros(int(np.sum(mask_too_small_values)))            


                if Highlight==True:
                    axe.plot(xx, yy_anySpin*np.sum(w[mask_anySpin]) / (yy_total*np.sum(w)),     color='k',        lw=2, zorder=20, alpha=1, ls=ls_)
                else:
                    axe.plot(xx, yy_anySpin*np.sum(w[mask_anySpin]) / (yy_total*np.sum(w)),     color='lightgray',      lw=2, zorder=6, alpha=1, ls=ls_)


                # remove labels and ticks  on X axe
                axe = layoutAxesNoXlabel(axe, nameX=nameX, nameY=nameY, setMinor=False, labelpad=5, fontsize=fs_l, labelSizeMajor=fs_major)
                for tick in axe.xaxis.get_major_ticks():
                    tick.tick1line.set_visible(False)
                    tick.tick2line.set_visible(False)
                    tick.label1.set_visible(False)
                    tick.label2.set_visible(False)

    if mode in [ 'MRR_fraction', 'notMRR_fraction', 'spinTwo_fraction', 'spinOne_fraction', 'm1spin_or_m2spin_fraction']:
        axe.set_yticks([ 0.5, 1])
    elif mode in ['MRR_CDF', 'spin_CDF']:
        axe.set_yticks([0.5, 0.75, 1])
        

            
    axe.set_xlim(xlim[0], xlim[1])
    axe.set_ylim(ylim[0], ylim[1])
    axe.grid(True)
    
    


    
    return axe 









In [ ]:




fig, ax = plt.subplots(3,3, figsize=(30,8), gridspec_kw={"height_ratios":[1, 1, 2.5], "width_ratios":[1, 1, 1]})#,\

fs_major=34


for ind_BPS, BPSmodel in enumerate(BPSnameslist[:]):

# for ind_BPS, BPSmodel in enumerate(['A']):
    
    print('running model %s'%BPSmodel)
    
    
    # CHIRP MASS 
    bw=0.85
    ylim_threshold = 0.0004       
    xparam = 'chirp_mass_LVK'
    plotYlog = False
    xlim, ylim = [0, 58], [0.0, 1.03]

#     ax[0,0] = plot_FC_distribution(axe=ax[0,0], xparam=xparam, BPSmodelName=BPSmodel, mode='notMRR_fraction',\
#                                   spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold) 
    ax[1,0] = plot_FC_distribution(axe=ax[1,0], xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_fraction',\
                                  spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold) 
    plotYlog = True
    ylim = [0.0003, 0.15]
    ax[2,0] = plot_FC_distribution(axe=ax[2,0], xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_PDF',\
                                  spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)



    
    
    
    ## MASS RATIO ##
    xparam = 'mass_ratio_LVK'
    ylim_threshold = 0.001
    bw = 0.01 
    plotYlog = False
    xlim, ylim = [0, 1], [0.0, 1.03]
#     ax[0,1] = plot_FC_distribution(axe=ax[0,1], xparam=xparam, BPSmodelName=BPSmodel, mode='notMRR_fraction',\
#                                   spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)
    ax[1,1] = plot_FC_distribution(axe=ax[1,1], xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_fraction',\
                                  spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold) 
    plotYlog = True
    ylim = [0.001, 9.15]
    ax[2,1] = plot_FC_distribution(axe=ax[2,1], xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_PDF',\
                                  spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)



    # chi spin  ##
    ylim_threshold = 0.001
    bw = 0.0051
    xparam = 'chi_of_spinning_BH'
    plotYlog = False
    xlim, ylim = [0.05, 1], [0.0, 1.03] 
#     ax[0,2] = plot_FC_distribution(axe=ax[0,2], xparam=xparam, BPSmodelName=BPSmodel, mode='notMRR_fraction',\
#                                   spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)
    
    ax[1,2] = plot_FC_distribution(axe=ax[1,2], xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_fraction',\
                                  spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold) 
    plotYlog = False
    ylim = [0.5, 1.03]
    ax[2,2] = plot_FC_distribution(axe=ax[2,2], xparam=xparam, BPSmodelName=BPSmodel, mode='MRR_CDF',\
                                  spin_threshold=0.05, bw=bw, xlim=xlim, ylim=ylim, plotYlog=plotYlog, ylim_threshold=ylim_threshold)




xx = np.linspace(1000,20000,100)
yy = np.linspace(1000,20000,100)
lw=12
fs = 30

# ax[2,1].plot(xx, yy,    color=colors_lighter[1], lw=lw, zorder=16, alpha=1, label=r'\textbf{MRR}')#, ls=ls_)
# ax[2,1].plot(xx, yy, color=colors_lighter[0], lw=lw, zorder=15, alpha=1, label=r'\textbf{non-MRR}')#, ls=ls_)
# ax[2,1].legend(fontsize=fs, frameon=False, loc='upper left')

    
plt.tight_layout()  
plt.subplots_adjust(wspace=0.2, hspace=0.1)



plt.savefig('./super_FC_split_panel2.pdf', transparent=False, bbox_inches="tight",  format='pdf')
plt.savefig('./super_FC_split_panel2.png', transparent=False, bbox_inches="tight", dpi=600, format='png')
plt.show()  











In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from corner import corner

# rvs(mean=None, cov=1, size=1, random_state=None)


y = multivariate_normal.rvs(size=100000, mean=[0,0])

corner(y);





# fig1 = plt.figure()
# ax = fig1.add_subplot(111)
# ax.plot(x, y)